   # Hello FLARE API

In this notebook, we go through the different commands of the FLARE API to show the syntax and usage of each.

### 1. Install NVIDIA FLARE and Provision an FL System

For this notebook, we will need a running NVFLARE project that we can connect to. Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions to set up an environment that has NVIDIA FLARE installed if you do not have one already.

Feel free to use an existing provisioned NVFLARE project if you have that available.

To try things out, you could set up and start a system in [POC mode](https://nvflare.readthedocs.io/en/main/getting_started.html#setting-up-the-application-environment-in-poc-mode), but note that you will need to use `new_insecure_session()` with the startup kit location as the only argument instead of `new_secure_session()`.

### 2. Start the FL System

Use the 'start.sh' scripts in the server and client startup kits to start the server and clients.

### 3. Connect to the FL System with the FLARE API and Get System Info

Use `new_secure_session()` to initiate a session connecting to the FL Server with the FLARE API. The necessary arguments are the username of the admin user you are using and the corresponding startup kit location (if you are using POC mode, you will need to use `new_insecure_session()` with the startup kit location as the only argument).

In the code example below, we get the `admin_user_dir` by concatenating the workspace root with the default directories that are created if you provision a project with a given project name. The assumption is that 'nvflare provision' has been run in a workspace set to '/workspace' to set up a project named `hello-example` with a server and two clients. You can change the workspace location and the project name to what applies for your environment if needed.

Note that if debug mode is not enabled, there is no output after initiating a session successfully, so instead we print the output of `get_system_info()`.

The `get_system_info()` command does not take any arguments, and returns a SystemInfo object consisting of server_info (server status and start time), client_info (each connected client and the last connect time for that client), and job_info (the list of current jobs with the job_id and app_name).

If you are unable to connect and initiate a session, make sure that your FL Server is running and that the configurations are correct with the right path to the admin startup kit directory.

In [22]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session

project_name = "hello-example"
username = "admin@nvidia.com"
workspace_root = "/workspace"
admin_user_dir = os.path.join(workspace_root, "workspace", project_name, "prod_00", username)

sess = new_secure_session(
    username=username,
    startup_kit_location=admin_user_dir
)
print(sess.get_system_info())

SystemInfo
server_info:
status: stopped, start_time: Thu Jan 26 15:12:01 2023
client_info:
site_a(last_connect_time: Thu Jan 26 19:47:29 2023)
site_b(last_connect_time: Thu Jan 26 19:47:30 2023)
job_info:



### 4. Submit Job
With a session successfully connected, you can use `submit_job()` to submit your job. You can change `path_to_example_job` to the location of the job you are submitting. Upon successful submission, the job_id is returned as a string.

In [23]:
path_to_example_job = "/workspace/NVFlare/examples/hello-numpy-sag"
job_id = sess.submit_job(path_to_example_job)
print(job_id + " was submitted")

81c48ff6-2b3a-46e8-a015-90f656a029c5 was submitted


### 5. Monitor Job

The command `monitor_job()` allows you to follow a job until the job is done.

By default, `monitor_job()` only has one required arguement, the `job_id` of the job you are waiting for, and the default behavior is to wait until the job is complete before returning a Return Code of `JOB_FINISHED`.

In order to follow along and see a more meaningful result and demonstrate some of the possibilities of using this function, the following cell contains a `sample_cb()` callback that keeps track of the number of times the callback is run and prints the `job_meta` the first three times and the final time before `monitor_job()` completes with every other call just printing a dot to save output space. This callback is just an example of what can be done with additional arguments and the `job_meta` information of the job that is being monitored. You can use logic to return a value other than True to conditionally stop `monitor_job()` and return `MonitorReturnCode.ENDED_BY_CB`.

In [25]:
from nvflare.fuel.flare_api.flare_api import Session

def sample_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3:
            print(job_meta)
            print(cb_kwargs["cb_run_counter"])
        else:
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True

sess.monitor_job(job_id, cb=sample_cb, cb_run_counter={"count":0})

{'name': 'hello-numpy-sag', 'job_folder_name': 'hello-numpy-sag', 'resource_spec': {}, 'deploy_map': {'hello-numpy-sag': ['@ALL']}, 'min_clients': 1, 'submitter_name': 'super@a.org', 'submitter_org': 'org_a', 'submitter_role': 'project_admin', 'job_id': '81c48ff6-2b3a-46e8-a015-90f656a029c5', 'submit_time': 1674780464.4234514, 'submit_time_iso': '2023-01-26T19:47:44.423451-05:00', 'start_time': '2023-01-26 19:47:46.114936', 'duration': 'N/A', 'status': 'RUNNING', 'job_deploy_detail': ['server: OK', 'site_a: OK', 'site_b: OK'], 'schedule_count': 1, 'last_schedule_time': 1674780465.0985308, 'schedule_history': ['2023-01-26 19:47:45: scheduled']}
{'count': 0}
{'name': 'hello-numpy-sag', 'job_folder_name': 'hello-numpy-sag', 'resource_spec': {}, 'deploy_map': {'hello-numpy-sag': ['@ALL']}, 'min_clients': 1, 'submitter_name': 'super@a.org', 'submitter_org': 'org_a', 'submitter_role': 'project_admin', 'job_id': '81c48ff6-2b3a-46e8-a015-90f656a029c5', 'submit_time': 1674780464.4234514, 'submi

<MonitorReturnCode.JOB_FINISHED: 0>

### 6. Get Job Meta

To get the job meta information, you can use the `get_job_meta()` command. The only argument required for this command is the job id for the job you are getting the meta for. The job meta information will be returned as an object.

In [26]:
sess.get_job_meta(job_id)

{'name': 'hello-numpy-sag',
 'job_folder_name': 'hello-numpy-sag',
 'resource_spec': {},
 'deploy_map': {'hello-numpy-sag': ['@ALL']},
 'min_clients': 1,
 'submitter_name': 'super@a.org',
 'submitter_org': 'org_a',
 'submitter_role': 'project_admin',
 'job_id': '81c48ff6-2b3a-46e8-a015-90f656a029c5',
 'submit_time': 1674780464.4234514,
 'submit_time_iso': '2023-01-26T19:47:44.423451-05:00',
 'start_time': '2023-01-26 19:47:46.114936',
 'duration': '0:00:47.791639',
 'status': 'FINISHED:COMPLETED',
 'job_deploy_detail': ['server: OK', 'site_a: OK', 'site_b: OK'],
 'schedule_count': 1,
 'last_schedule_time': 1674780465.0985308,
 'schedule_history': ['2023-01-26 19:47:45: scheduled']}

### 7. List Jobs

To get the information for what jobs have been submitted to the server, you can use the `list_jobs()` command. If no arguments are included, the defaults are False for both "detailed" and "all".

Setting "detailed" to True will return more detailed information about each job.

Setting "all" to True will return all the jobs that have been submitted to the server regardless of the submitting user. By default, only the jobs that have been submitted by the same user will show up for `list_jobs()`.

In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))

list_jobs_output = sess.list_jobs()
print(format_json(list_jobs_output))


In [ ]:

list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

### 8. Download Job Result

The `download_job_result()` command downloads the job result to the "download_dir" (this is usually set in fed_admin.json in the startup directory of the admin_user_dir used when launching the FLARE API Session, and this value is relative to the admin_user_dir). This command only has one required arguement, the `job_id` of the job result to download.

In [ ]:
sess.download_job_result(job_id)

### 9. Clone Jobs

To clone an existing job, you can use the `clone_job()` command. The `clone_job()` command only has one required arguement, the `job_id` of the job to clone.

In [ ]:
sess.clone_job(job_id)

### 10. Abort Job

If training gets stuck or there is another reason to stop a running job, you can use the `abort_job()` command. The `abort_job()` command only has one required arguement, the `job_id` of the job to abort, and None is returned upon successfully issuing the command.

In [40]:
sess.abort_job(job_id)

None


### 11. Delete Job

You should be able to see the output in the terminals where you are running your FL Server and Clients when you submitted the job. You can also use `monitor_job()` to follow along and give you updates on the progress until the job is done.

By default, `delete_job()` only has one required argument, the `job_id` of the job you are waiting for, and the default behavior is to return nothing if the command is successful. The command will raise a `JobNotDone` exception if the job is still running. 

In [20]:
sess.delete_job(job_id)

### 12. All Other Commands

For all other commands that do not have a specific FLARE API implementation, using ``api.do_command()`` should provide what the FLAdminAPI would have returned in the raw result.

In [14]:
sess.api.do_command("list_sp")

{'status': <APIStatus.SUCCESS: 'SUCCESS'>,
 'details': "{'primary_sp': {'sp_end_point': 'server1:8002:8003', 'service_session_id': 'ebc6125d-0a56-4688-9b08-355fe9e4d61a', 'primary': True, 'state': 'online'}, 'sp_list': [{'sp_end_point': 'server1:8002:8003', 'service_session_id': 'ebc6125d-0a56-4688-9b08-355fe9e4d61a', 'primary': True, 'state': 'online'}], 'system': 'ready'}",
 'data': {'primary_sp': {'sp_end_point': 'server1:8002:8003',
   'service_session_id': 'ebc6125d-0a56-4688-9b08-355fe9e4d61a',
   'primary': True,
   'state': 'online'},
  'sp_list': [{'sp_end_point': 'server1:8002:8003',
    'service_session_id': 'ebc6125d-0a56-4688-9b08-355fe9e4d61a',
    'primary': True,
    'state': 'online'}],
  'system': 'ready'}}